In [ ]:
# 1. Access the twitter api to download a bunch of tweets about san fransisco. 
#    I ran this from the command line to pipe data to a file

import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#print 'Begin script...'

consumer_key = "IyhOZXYvlps1ABWB6UWMBmVsh"
consumer_secret =  "SIFfJp78RCcBS3JmECj7ZS4MuJ8apYqtNMbAC3Zy7xd2Szg5Wq"
access_token = '1411256130-wJk78NyXkZLJlPVsGCHq8mgtX0wHcDanc73fy0k'
access_token_secret = 'sNPVmByioAhvlcQXHVmg2wd5lU3RJZF5drDcrsx0kaOCi'


#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):
 
    def on_data(self, data):
        print data
        return True

    def on_error(self, status):
        print status

#This handles Twitter authetification and the connection to Twitter Streaming API
mylistener = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
mystream = Stream(auth, mylistener)

#print 'starting...'
mystream.filter(track=['sanfrancisco'])

In [ ]:
#2.  What other hashtags are the most common in tweets about san francisco?  
#    What's the average number of followers for people tweeting about SF?  

In [40]:
#4.  Parse out the json structure of each tweet and load into a Pandas dataframe

import pprint
import json
import pandas as pd

rawfile = open('sf_tweets.txt', 'r')

languages_l, followers_l, hashtags_l = [], [], []

for line in rawfile:

    cleanline = line.strip()
    if cleanline == '':
        continue 

    tweetfull = json.loads(line.strip())
    #pprint.pprint(tweetfull)
    
    hashtags = []
    for tag in tweetfull['entities']['hashtags']:
        hashtags.append(tag['text'])
    #print hashtags
    language = tweetfull['user']['lang']
    #print language
    followers = tweetfull['user']['followers_count']

    languages_l.append(language)
    followers_l.append(followers)
    hashtags_l.append(hashtags)
    
rawfile.close()
df = pd.DataFrame({'languages': languages_l, 'followers': followers_l, 'hashtags': hashtags_l})
    
print df.head(10)
print df.info()


   followers                                           hashtags languages
0        489                 [Oracle, Unix, SanFrancisco, jobs]        en
1        552  [SanFrancisco, Sales, Job, Jobs, Hiring, Caree...        en
2         15                             [Travel, SanFrancisco]        en
3      28226  [SanFrancisco, Nike, Broncos, PeytonManning, J...        en
4        102  [MondayMotivation, jenselter, seltering, gymti...        en
5       1760                                     [sanfrancisco]        en
6       1760                                     [sanfrancisco]        en
7       1760                        [FeelTheBern, sanfrancisco]        en
8       1760                                     [sanfrancisco]        en
9       1760                                     [sanfrancisco]        en
<class 'pandas.core.frame.DataFrame'>
Int64Index: 245 entries, 0 to 244
Data columns (total 3 columns):
followers    245 non-null int64
hashtags     245 non-null object
languages    245 

In [45]:
#5.  Answer my quesions about the hashtags and average followers 

import pprint

print df.followers.mean()

tag_frequency = {}

for taglist in df.hashtags:
    for tag in taglist:
        tag_frequency[tag.lower()] = tag_frequency.get(tag.lower(), 0) + 1


df_frequency = pd.DataFrame.from_dict(tag_frequency, orient='index')

df_frequency.sort(columns=0, inplace=True, ascending=False)
print df_frequency.head(15)
print df_frequency.info()




1537.05306122
                     0
sanfrancisco       215
job                 70
jobs                62
hiring              42
careerarc           25
travel              14
engineer            13
amazonfulfillment   12
sf                   9
sales                8
amazon               8
reporting            8
software             8
california           7
music                5
<class 'pandas.core.frame.DataFrame'>
Index: 288 entries, sanfrancisco to summer
Data columns (total 1 columns):
0    288 non-null int64
dtypes: int64(1)
memory usage: 4.5+ KB
None


In [50]:
#6.  Use Bokeh to make a vizualization of the data 

import numpy as np
from bokeh.plotting import figure, output_notebook,show,vplot

p1 = figure(title="Followers Histogram",x_axis_label = 'Followers',y_axis_label = 'Count')
#construct the histogram
hist, edges = np.histogram(df.followers.values, density=True, bins=50)
#Construct your x axis
x = np.linspace(df.followers.min(),df.followers.max(),100)
#add the bars, scaling the value to the full count of people
p1.quad(top=hist*len(df.followers), bottom=0, left=edges[:-1], right=edges[1:],line_color='black')


p2 = figure(title="Followers Histogram",x_axis_label = 'Followers',y_axis_label = 'Count')
hist, edges = np.histogram(df.followers.values, density=True, bins=50)
x = np.linspace(df.followers.min(),df.followers.max(),100)
p2.quad(top=hist*len(df.followers), bottom=0, left=edges[:-1], right=edges[1:],line_color='black')


show(vplot(p1,p2))



ImportError: cannot import name vplot